In [50]:
from urllib.request import urlopen
from IPython.display import display, HTML
from bs4 import BeautifulSoup, Comment
import pdb
import requests
import re
import pandas as pd
import os

NBA_URL = 'https://www.basketball-reference.com'
NBA_SOUP = BeautifulSoup(urlopen(NBA_URL), 'html.parser')


def find_in_page_text(page, class_=None, text_=None):
    return page.find(class_, text=text_)

def find_in_page_id(page, class_=None, id_=None):
    return page.find(class_, id = id_)

def go_to_page(url, page, class_=None, text_=None):
    link = page.find(class_, text=text_)
    new_url = url + str(link.get('href'))
    return new_url, BeautifulSoup(urlopen(new_url), 'html.parser')


def get_league_soup():
    nav_bar = nba_site.find('div', attrs={'id': 'nav'})
    league = nav_bar.find('li', attrs = {'id': 'header_leagues'})
    league_url = nba_url + str(league.find('a').get('href'))

    league_soup = urlopen(league_url)
    league_soup= BeautifulSoup(league_soup, 'html.parser')
    return league_soup


#Input: League page: basketball-reference.com/leagues
#Output: Dictionary of Key: Year Value: basketball-reference.com/leagues/NBA_YEAR.html
#Retrieves the basic statistics of e
def get_seasons_dict(league_soup=None):
    if league_soup == None:
        nav_bar = NBA_SOUP.find('div', attrs={'id': 'nav'})
        nav_bar_league = nav_bar.find('li', attrs = {'id': 'header_leagues'})
        league_url = nba_url + str(nav_bar_league.find('a').get('href'))
        league_soup = BeautifulSoup(urlopen(league_url), 'html.parser')
    
    NBA = find_in_page_text(league_soup, 'span', "Seasons")
    NBA_by_year = (NBA.parent.find_all('li'))
    url_by_season = dict()
    

    for i in NBA_by_year:
        #We only consider NBA. Not ABA/BBA
        if "NBA" not in str(i):
            continue
        year = [s for s in str(i.find("a").get("href")) if s.isdigit()]
        year = int(''.join(year))
        url_by_season[year]= nba_url + str(i.find('a').get('href'))
    return url_by_season

def get_team_stats_in_year(year, year_soup):
    categories = []

    categories.append("all_team-stats-base")
    categories.append("all_opponent-stats-base")
    categories.append("all_team-stats-per_poss")
    categories.append("all_opponent-stats-per_poss")
    categories.append("all_misc_stats")

    for category in categories:
        for comments in year_soup.find("div", id=category)\
            .findAll(text=lambda text:isinstance(text, Comment)):
            extracted_comment = comments.extract()
            commented_page = BeautifulSoup(extracted_comment)
            if category == "all_misc_stats":
                headers = [th.getText() for th in commented_page.findAll('tr', limit=2)[1].findAll('th')]
            else:
                headers = [th.getText() for th in commented_page.findAll('tr', limit=2)[0].findAll('th')]
                
            headers = headers[1:]
            rows = commented_page.findAll('tr')[1:]
            player_stats = [[td.getText() for td in rows[i].findAll('td')] \
                for i in range(len(rows))]
            stats = pd.DataFrame(player_stats, columns = headers)
            directory = "Data/" + str(year)
            if (not os.path.exists(directory)):
                 os.mkdir(directory)
            stats.to_pickle(directory + "/" + category + ".pkl")
def get_team_stats_in_all_years():
    league_soup = get_league_soup()
    seasons = get_seasons_dict(league_soup)
    for year in seasons.keys():
        year_soup = BeautifulSoup(urlopen(seasons[year]) , 'html.parser')
        get_team_stats_in_year(year, year_soup)
    return



In [64]:
get_team_stats_in_all_years()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/jlee/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3296, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-64-04613a082c8e>", line 1, in <module>
    get_team_stats_in_all_years()
  File "<ipython-input-63-7a3b5857285d>", line 34, in get_team_stats_in_all_years
    get_team_stats_in_year(year, year_soup)
  File "<ipython-input-63-7a3b5857285d>", line 27, in get_team_stats_in_year
    os.mkdir(directory)
FileNotFoundError: [Errno 2] No such file or directory: 'Data/2019'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/jlee/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2033, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'FileNotFoundError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

FileNotFoundError: [Errno 2] No such file or directory: 'Data/2019'